In [1]:
!pip install fhirpathpy
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from fhirpathpy import evaluate

In [3]:
patient = {
  "resourceType": "Patient",
  "id": "example",
  "name": [
    {
      "use": "official",
      "given": [
        "Peter",
        "James"
      ],
      "family": "Chalmers"
    },
    {
      "use": "usual",
      "given": [
        "Jim"
      ]
    },
    {
      "use": "maiden",
      "given": [
        "Peter",
        "James"
      ],
      "family": "Windsor",
      "period": {
        "end": "2002"
      }
    }
  ]
}

Determine if we are dealing with a Patient

In [4]:
result = evaluate(patient, "resourceType = 'Patient'", [])
print(result)


[True]


In [5]:
result = evaluate(patient, "Patient.name.given.empty()", [])
print(result)

[False]


In [6]:
result = evaluate(patient, "name.where(use='official').given | name.where(use='official').family", [])
print(result)

['Peter', 'James', 'Chalmers']


In [7]:
result = evaluate(patient, "(name.where(use='official').given | name.where(use='official').family).count()", [])
print(result)

[3]


In [16]:
result = evaluate(patient, "name.where(use='official').union(name.where(use='maiden'))", [])
print(result)

[{'use': 'official', 'given': ['Peter', 'James'], 'family': 'Chalmers'}]


In [17]:
result = evaluate(patient, "(name.where(use='maiden').exists()", [])
print(result)

[True]


In [18]:
result = evaluate(patient, "(name.given.where($this > 'Pa')", [])
print(result)

['Peter', 'Peter']


In [21]:
# Evaluating FHIRPath
result = evaluate(patient, "Patient.name.where(use='official').given.first()", [])

In [22]:
print(result)

['Peter']


In [23]:
result = evaluate(patient, "Patient.name.where(use = 'official').family", [])
print(result)

['Chalmers']


In [24]:
result = evaluate(patient, "Patient.name.given", []) 
print(result)

['Peter', 'James', 'Jim', 'Peter', 'James']


In [25]:
result = evaluate(patient, "name.given", []) 
print(result)

['Peter', 'James', 'Jim', 'Peter', 'James']


Check for the Existence of a Birth Date

In [26]:
result = evaluate(patient, "Patient.birthDate.exists()", []) 
print(result)

[False]


In [28]:
result = evaluate(patient, "Patient.name.period.exists())", []) 
print(result)

[True]


In [33]:
result = evaluate(patient, "Patient.name.where(use = 'maiden').given.last()", [])
print(result)

['James']


In [41]:
result = evaluate(patient, "Patient.name.where(use = 'official').first().given.join(' ') + ' ' + Patient.name.where(use = 'official').first().family", []) 
print(result)

['Peter James Chalmers']


In [42]:
result = evaluate(patient, "Patient.name.given.distinct()", []) 
print(result)

['Peter', 'James', 'Jim']


In [44]:
result = evaluate(patient, "Patient.name.where(use = 'official').first().given.last()", [])
print(result)

['James']


In [47]:
obs = {
  "resourceType": "Observation",
  "id": "example",
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "vital-signs",
          "display": "Vital Signs"
        }
      ]
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "85354-9",
        "display": "Blood pressure panel with all children optional"
      }
    ],
    "text": "Blood pressure systolic & diastolic"
  },
  "subject": {
    "reference": "Patient/example",
    "display": "Peter Chalmers"
  },
  "effectiveDateTime": "2020-03-15T09:30:10+01:00",
  "issued": "2020-03-15T09:45:10+01:00",
  "component": [
    {
      "code": {
        "coding": [
          {
            "system": "http://loinc.org",
            "code": "8480-6",
            "display": "Systolic blood pressure"
          }
        ]
      },
      "valueQuantity": {
        "value": 120,
        "unit": "mmHg",
        "system": "http://unitsofmeasure.org",
        "code": "mm[Hg]"
      }
    },
    {
      "code": {
        "coding": [
          {
            "system": "http://loinc.org",
            "code": "8462-4",
            "display": "Diastolic blood pressure"
          }
        ]
      },
      "valueQuantity": {
        "value": 80,
        "unit": "mmHg",
        "system": "http://unitsofmeasure.org",
        "code": "mm[Hg]"
      }
    }
  ]
}


In [48]:
result = evaluate(obs, "component.valueQuantity.value", [])
print(result)

[120, 80]


In [49]:
result = evaluate(obs, "Observation.status", [])
print(result)

['final']


In [50]:
result = evaluate(obs, "Observation.effectiveDateTime", [])
print(result)

['2020-03-15T09:30:10+01:00']


In [51]:
result = evaluate(obs, "Observation.code.text", [])
print(result)

['Blood pressure systolic & diastolic']


In [52]:
result = evaluate(obs, "Observation.component.where(code.coding.code = '8480-6').valueQuantity.value", [])
print(result)

[120]


In [53]:
result = evaluate(obs, "Observation.component.where(code.coding.code = '8480-6').code.coding.display", [])
print(result)

['Systolic blood pressure']


In [54]:
result = evaluate(obs, "Observation.subject.display", [])
print(result)

['Peter Chalmers']


In [55]:
result = evaluate(obs, "Observation.issued", [])
print(result)

['2020-03-15T09:45:10+01:00']


In [56]:
result = evaluate(obs, "Observation.category.coding.code", [])
print(result)

['vital-signs']


In [57]:
result = evaluate(obs, "Observation.component.where(code.coding.code = '8480-6').valueQuantity.value.toString() + '/' + Observation.component.where(code.coding.code = '8462-4').valueQuantity.value.toString()", [])
print(result)

['120/80']


In [58]:
bundle = {
  "resourceType": "Bundle",
  "id": "bundle-example",
  "type": "collection",
  "entry": [
    {
      "resource": {
        "resourceType": "Observation",
        "id": "obs1",
        "status": "final",
        "code": {
          "coding": [
            {
              "system": "http://loinc.org",
              "code": "85354-9",
              "display": "Blood pressure panel"
            }
          ],
          "text": "Blood Pressure"
        },
        "subject": {
          "reference": "Patient/123"
        },
        "effectiveDateTime": "2025-02-25T10:00:00+00:00",
        "component": [
          {
            "code": {
              "coding": [
                {
                  "system": "http://loinc.org",
                  "code": "8480-6",
                  "display": "Systolic blood pressure"
                }
              ]
            },
            "valueQuantity": {
              "value": 120,
              "unit": "mmHg"
            }
          },
          {
            "code": {
              "coding": [
                {
                  "system": "http://loinc.org",
                  "code": "8462-4",
                  "display": "Diastolic blood pressure"
                }
              ]
            },
            "valueQuantity": {
              "value": 80,
              "unit": "mmHg"
            }
          }
        ]
      }
    },
    {
      "resource": {
        "resourceType": "Observation",
        "id": "obs2",
        "status": "final",
        "code": {
          "coding": [
            {
              "system": "http://loinc.org",
              "code": "8867-4",
              "display": "Heart rate"
            }
          ],
          "text": "Heart Rate"
        },
        "subject": {
          "reference": "Patient/123"
        },
        "effectiveDateTime": "2025-02-25T10:05:00+00:00",
        "valueQuantity": {
          "value": 72,
          "unit": "beats/minute"
        }
      }
    }
  ]
}


Extract the status of all observations

In [59]:
result = evaluate(bundle, "Bundle.entry.resource.status", [])
print(result)

['final', 'final']


In [60]:
result = evaluate(bundle, "entry.resource.status", [])
print(result)

['final', 'final']


Get the IDs of observations with a code text equal to 'Blood Pressure'

In [61]:
result = evaluate(bundle, "Bundle.entry.resource.where(code.text = 'Blood Pressure').id", [])
print(result)

['obs1']


Retrieve the systolic blood pressure value (from the component with code '8480-6')

In [62]:
result = evaluate(bundle, "Bundle.entry.resource.where(component.exists()).component.where(code.coding.code = '8480-6').valueQuantity.value", [])
print(result)

[120]


Retrieve the diastolic blood pressure value (from the component with code '8462-4')

In [63]:
result = evaluate(bundle, "Bundle.entry.resource.where(component.exists()).component.where(code.coding.code = '8462-4').valueQuantity.value", [])
print(result)

[80]


In [64]:
result = evaluate(bundle, "Bundle.entry.resource.ofType(Observation)", [])
print(result)

[{'resourceType': 'Observation', 'id': 'obs1', 'status': 'final', 'code': {'coding': [{'system': 'http://loinc.org', 'code': '85354-9', 'display': 'Blood pressure panel'}], 'text': 'Blood Pressure'}, 'subject': {'reference': 'Patient/123'}, 'effectiveDateTime': '2025-02-25T10:00:00+00:00', 'component': [{'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood pressure'}]}, 'valueQuantity': {'value': 120, 'unit': 'mmHg'}}, {'code': {'coding': [{'system': 'http://loinc.org', 'code': '8462-4', 'display': 'Diastolic blood pressure'}]}, 'valueQuantity': {'value': 80, 'unit': 'mmHg'}}]}, {'resourceType': 'Observation', 'id': 'obs2', 'status': 'final', 'code': {'coding': [{'system': 'http://loinc.org', 'code': '8867-4', 'display': 'Heart rate'}], 'text': 'Heart Rate'}, 'subject': {'reference': 'Patient/123'}, 'effectiveDateTime': '2025-02-25T10:05:00+00:00', 'valueQuantity': {'value': 72, 'unit': 'beats/minute'}}]


Extract the heart rate value (observation with code '8867-4')

In [91]:
result = evaluate(bundle, "Bundle.entry.resource.where(code.coding.code = '8867-4').valueQuantity.value", [])
print(result)

[72]


In [103]:
result = evaluate(bundle, "entry.resource.where(resourceType = 'Observation').where(component.valueQuantity.value.exists($this > 100))", [])
print(result)

[{'resourceType': 'Observation', 'id': 'obs1', 'status': 'final', 'code': {'coding': [{'system': 'http://loinc.org', 'code': '85354-9', 'display': 'Blood pressure panel'}], 'text': 'Blood Pressure'}, 'subject': {'reference': 'Patient/123'}, 'effectiveDateTime': '2025-02-25T10:00:00+00:00', 'component': [{'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood pressure'}]}, 'valueQuantity': {'value': 120, 'unit': 'mmHg'}}, {'code': {'coding': [{'system': 'http://loinc.org', 'code': '8462-4', 'display': 'Diastolic blood pressure'}]}, 'valueQuantity': {'value': 80, 'unit': 'mmHg'}}]}]


Must select the component we want

In [107]:
result = evaluate(bundle, "entry.resource.where(resourceType = 'Observation').where(component.valueQuantity.value.exists($this > 100)).select(component.where(valueQuantity.value > 100))", [])
print(result)

[{'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood pressure'}]}, 'valueQuantity': {'value': 120, 'unit': 'mmHg'}}]


In [108]:
result = evaluate(bundle, "entry.resource.component.valueQuantity.value.where($this > 100)", [])
print(result)

[120]


Are all the component blood pressure values greater than or equal to 80?

In [109]:
result = evaluate(bundle, "entry.resource.where(resourceType = 'Observation').where(component.exists() and (component.valueQuantity.value.all(($this >= 80))", [])
print(result)

[{'resourceType': 'Observation', 'id': 'obs1', 'status': 'final', 'code': {'coding': [{'system': 'http://loinc.org', 'code': '85354-9', 'display': 'Blood pressure panel'}], 'text': 'Blood Pressure'}, 'subject': {'reference': 'Patient/123'}, 'effectiveDateTime': '2025-02-25T10:00:00+00:00', 'component': [{'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood pressure'}]}, 'valueQuantity': {'value': 120, 'unit': 'mmHg'}}, {'code': {'coding': [{'system': 'http://loinc.org', 'code': '8462-4', 'display': 'Diastolic blood pressure'}]}, 'valueQuantity': {'value': 80, 'unit': 'mmHg'}}]}]


Get all effective date and time values from the observations

In [110]:
result = evaluate(bundle, "Bundle.entry.resource.effectiveDateTime", [])
print(result)

['2025-02-25T10:00:00+00:00', '2025-02-25T10:05:00+00:00']


Return the subject reference (patient identifier) for all observations

In [112]:
result = evaluate(bundle, "Bundle.entry.resource.subject.reference.distinct()", [])
print(result)

['Patient/123']


Retrieve the unit for the systolic measurement

In [113]:
result = evaluate(bundle, "Bundle.entry.resource.where(component.exists()).component.where(code.coding.code = '8480-6').valueQuantity.unit", [])
print(result)

['mmHg']


Count the number of Observation resources in the bundle

In [114]:
result = evaluate(bundle, "Bundle.entry.resource.count()", [])
print(result)

[2]


Concatenate the first observation’s ID with its effective date/time 

In [115]:
result = evaluate(bundle, "Bundle.entry.resource.id.first() + ' ' + entry.resource.effectiveDateTime.first()", [])
print(result)

['obs1 2025-02-25T10:00:00+00:00']


# Make the GET request to retrieve the bundle of Patients

In [116]:
import requests
from requests.auth import HTTPBasicAuth

# Define the URL and headers for the GET request

In [117]:
url = "http://127.0.0.1:8080/csp/healthshare/demo/fhir/r4/Patient"
headers = {
    "Accept": "*/*",
    "content-type": "application/fhir+json",
    "Accept-Encoding": "gzip, deflate, br",
    "Prefer": "return=representation"
}


In [118]:
response = requests.get(url, headers=headers, auth=HTTPBasicAuth('_System', 'ISCDEMO'))

# Check if the request was successful
if response.status_code == 200:
    bundle = response.json()
    print("Bundle retrieved successfully!")
    print(f"Total Patients: {len(bundle.get('entry', []))}")
    
    # You can now use FHIRPath or further analyze the bundle as needed.
    # For example, print out the first patient's ID:
    if bundle.get('entry'):
        first_patient = bundle['entry'][0]['resource']
        print("First Patient ID:", first_patient.get('id'))
else:
    print("Error retrieving bundle:", response.status_code)
    print(response.text)

Bundle retrieved successfully!
Total Patients: 126
First Patient ID: 2


Let's find the id of each patient resource in the bundle we just obtained.

In [119]:
result = evaluate(bundle, "Bundle.entry.resource.id", [])
print(result)


['2', '3', '6', '1', '4', '5', '7', '8', '9', '641', '1883', '1752', '2244', '2015', '2401', '3967', '5959', '4976', '4347', '4606', '5587', '6883', '6055', '11092', '10552', '12444', '12260', '13257', '14976', '14522', '16887', '17268', '17674', '17427', '17111', '19317', '19948', '19710', '19539', '21629', '19582', '22341', '24850', '23520', '23300', '24767', '22806', '24453', '26343', '26135', '25927', '25616', '27137', '28643', '30037', '28864', '31200', '28587', '30530', '32203', '31979', '34144', '32324', '34747', '34268', '34613', '36115', '37324', '38105', '38528', '38855', '37712', '39419', '39752', '37011', '40810', '41691', '40360', '42737', '45278', '42979', '43830', '46599', '46375', '46996', '47813', '48554', '50033', '50604', '48461', '51476', '49070', '52327', '52036', '50806', '51042', '52967', '54047', '52118', '53416', '54424', '55664', '56025', '58130', '57935', '57369', '57085', '58539', '59584', '58268', '57728', '58961', '59969', '61759', '64235', '70082', '70083

Find all the female patients in the FHIR repository

In [123]:
result = evaluate(bundle, "Bundle.entry.resource.where(gender = 'female').count()", [])
print(result)

[63]


In [124]:
url = "http://127.0.0.1:8080/csp/healthshare/demo/fhir/r4/Condition?subject=Patient/2"
headers = {
    "Accept": "*/*",
    "content-type": "application/fhir+json",
    "Accept-Encoding": "gzip, deflate, br",
    "Prefer": "return=representation"
}

In [125]:
response = requests.get(url, headers=headers, auth=HTTPBasicAuth('_System', 'ISCDEMO'))

# Check if the request was successful
if response.status_code == 200:
    bundle = response.json()
    print("Bundle retrieved successfully!")
    # Define the FHIRPath expression:
    # This expression filters Bundle entries for Condition resources where at least one coding in clinicalStatus has code 'active'
    expression1 = (
        "Bundle.entry.where(resource.resourceType = 'Condition' and "
        "resource.clinicalStatus.coding.where(code = 'active').exists()).count()"
    )
    
    expression2 = (
        "Bundle.entry.where(resource.resourceType = 'Condition' and "
        "resource.clinicalStatus.coding.where(code = 'resolved').exists()).count()"
    )
    
    # Evaluate the FHIRPath expression on the bundle
    active_count = evaluate(bundle, expression1, [])
    resolved_count = evaluate(bundle, expression2, [])
    print(f"Patient two has {active_count} active conditions and {resolved_count} resolved conditions.")
else:
    print("Failed to retrieve bundle")

Bundle retrieved successfully!
Patient two has [2] active conditions and [8] resolved conditions.


Problems with Types

In [129]:
ptdec = {
  "resourceType": "Patient",
  "id": "fake-patient-example",
  "name": [
    {
      "use": "official",
      "family": "Doe",
      "given": ["Jane", "Elizabeth"]
    }
  ],
  "address": [
    {
      "use": "home",
      "line": ["1234 Main Street"],
      "city": "Exampleville",
      "state": "CA",
      "postalCode": "90001",
      "country": "USA"
    }
  ],
  "deceasedBoolean" : "true",
  "birthDate": "1980-01-15"
}

In [133]:
result = evaluate(ptdec, "deceased.exists().select($this.type())", [])
print(result)

[{'name': 'Boolean', 'namespace': 'System'}]


In [134]:
result = evaluate(ptdec, "deceasedBoolean.exists()", [])
print(result)

[True]


In [135]:
result = evaluate(ptdec, "deceasedDateTime.exists()", [])
print(result)

[False]
